#  Linear Regression

## 1.Homework

Learning from data [第二周作业](http://www.work.caltech.edu/homework/hw2.pdf)的5-7问题. 使用线性回归, 探索分类问题.

## 分析问题

题目简要:

- 数据集:
    - D: 输入空间 X: [-1, 1] x [-1, 1] 均匀分布产生.
        - 每个输入样本特征 2个, 每个特征参数范围为 [-1, 1]
        - 样本数目为 N
- Target Function: f
    - 随机生成一条线, 随机从 [-1, 1] x [-1, 1] 空间取两个点, 通过这两个随机点的直线作为 target function.
    - f 一边的样本划分为 +1 类别, 在另一边的样本划分为 -1 类别. 自己设定.
    - 数据集 D 中的样本 x_n 进行 类别划分. (这样就生成完成数据集 (X, Y))
- 模型: 线性回归模型
- 问题5:
    - 试验次数 1000
    - 每一次试验
        - 数据集样本数目 N, 数据集按上面的数据集部分产生 
        - 使用线性回归 找到 final function g, 评估 E{in} = 分类不准确率, 1000次试验取平均
        - 哪一个和问题中给出的接近.
- 问题6:
    - 试验 1000次
    - 每次试验
        - 随机产生 1000个 新的点, 
        - 用这些点计算 out-of-sample error E{out}(g), 最终假设使用 问题5 中得到的 g
      - 取平均值, 看和问题中给出的哪一个选项接近  
- 问题7: 
    - 数据集样本数目 N=10
    - 1000 次 run
    - 每一次run
        - 使用 是 linear regression 得到的权重, 作为 PLA 算法的初始权重
        - 记录 PLA 的收敛迭代次数 
    - 取 1000次运行 迭代次数的平均值, 看问题中给出的选项, 哪一个更接近.
- 线性回归:
    - [LFD第三章笔记:线性模型 · Anifacc](https://anifacc.github.io/machinelearning/learningfromdata/2017/09/14/lfd-ch03-linear-model/)
    - w{lin} = (X^T X)^-1 X^T y

## Specification & Design

### 1 产生数据集D

generalize_dataset()

1. 产生 N 个样本xi 的 特征(2d [-1, 1] x [-1, 1])
2. 产生 N 个样本xi 对应的标签 yi(根据目标函数f得到)

伪代码:

```
1. Generalize N samples in 2d, x1 = (x11, x12), ..., xN = (xN1, xN2) in [-1, 1]x[-1, 1]
2. Through target function get the true labels of the samples Y1, Y2, ..., YN
```

### 2 产生目标函数 

target_f()

- 流程
    1. 随机产生两个数据样本, 2d, [-1, 1] x [-1, 1]
    2. 得到 tagrget function f
    3. 划分类别
- 伪代码

```
    1. Generalize 2 points in 2 dimension [-1, 1] x [-1, 1]
        - (x1, y1)
        - (x2, y2)
    2. Through the 2 points calculating slope
        - slope = (y2 - y1) / (x2 - x1)
        - f: y = slope * (x - x1) + y1
    3. if yi > slope * (xi - x1) + y1:
        - then the label of the sample (xi, yi) is +1 
        + else:
            - the label of the sample (xi, yi) is -1
```

### 3 Final Hypothesis g

hypothesis_g()

1. 使用线性回归 得到权重
2. 得到 g

线性回归伪代码:

```
1. X = [x1^T, x2^T, ..., xN^T]^ T
2. w{lin} = (X^T X)^-1 X^T y
3. g = w^T x
```

### 问题5

流程

- one experiment
    1. Generalize dataset D, random data in 2d [-1, 1] x [-1, 1] space, x1, x2, ..., xN
    2. Genearlize the labels y1, y2, ..., yN of N samples
    3. Training data
        - Linear regression : 权重
        - get finanl hypothesis g
        - sign(g)
        - Calculate E{in}
- Repeat 1000 experiments
    - Calculate average in-sample error E[in] average
 
伪代码:

```
for i = 1, 2, ..., N_expriment,
    run experiment:
    1. through generalize_dataset() get dataset D (N samples)
    2. throuth target_f() get the labels of the dataset
    3. linear regression get weights
    4. through hypothesis_g() get predict label
    5. calculate E{in}_i = [[g != y]] / N
    6. sum_E =  sum(E_{in}_i)

calculate average E_{in} = sum_E / N_expriment
   
```

In [1]:
import numpy as np
import random

In [2]:
# 生成数据集
def generate_dataset(data_size):
    # A List of lists 
    dataset = [[1., random.uniform(-1, 1), random.uniform(-1, 1)]
                for i in range(data_size)]
    return dataset

def target_f(feature, point1, point2):
    # feature: [1, x1, x2]
    x, y = feature[1], feature[2]
    x1, y1 = point1
    x2, y2 = point2
    slope = (y2 - y1) / (x2 - x1)
    
    return 1 if y > (slope * (x - x1) + y1) else -1

In [3]:
# 假设集判断
def hypothesis(feature, weights):
    return np.sign(np.dot(feature, weights))

In [4]:
# 模型训练
def train(training_set, p1, p2):
    data_matrix = np.array(training_set)
    target_vector = np.array([[target_f(point, p1, p2)] for point in training_set])
    
    pinv = np.linalg.pinv(data_matrix)
    
    weights = np.dot(pinv, target_vector)
    return weights

In [5]:
# 测试集测试
def test(testing_set, p1, p2, weights):
    mismatches = 0
    
    for feature in testing_set:
        if hypothesis(feature, weights) != target_f(feature, p1, p2):
            mismatches += 1
            
    mismatches /= float(len(testing_set))
    
    return mismatches

In [10]:
def experiments(train_size, test_size, num_exp):
    training_size  = train_size
    testing_size = test_size
    
    avg_err_in = 0
    avg_err_out = 0
    
    for i in xrange(num_exp):
        p1 = (random.uniform(-1, 1), random.uniform(-1, 1))
        p2 = (random.uniform(-1, 1), random.uniform(-1, 1))
        
        training_set = generate_dataset(training_size)
        testing_set = generate_dataset(testing_size)
        
        final_weights = train(training_set, p1, p2)
        
        err_in = test(training_set, p1, p2, final_weights)
        avg_err_in += err_in
        
        err_out = test(testing_set, p1, p2, final_weights)
        avg_err_out += err_out
    
    avg_err_in /= float(num_exp)
    avg_err_out /= float(num_exp)
    
    return avg_iter, avg_err_in, avg_err_out

In [21]:
def pla(training_set, p1, p2, weights=None):
    misclassified = []
    iterations = 0
    
    if weights is None:
        weights = np.array([[0., 0., 0.]]).T
    
    while True:
        for feature in training_set:
            true_label = target_f(feature, p1, p2)
            # 尚未正确分类的样本判断
            if hypothesis(feature, weights) != true_label:
                misclassified.append((feature, true_label))
            
        if not misclassified:
            break
        else:
            iterations += 1
            # 随机从错误分类的样本中选择一个样本, 进行权重更新
            x, y = random.choice(misclassified)
            # update w(t+1) = w(t) + y*x*
            adapt = np.array([x]).T * y
            weights += adapt
            misclassified = []
           
    return iterations, weights

In [16]:
data_size = 100
test_size = 1000

num_expriment = 1000

avg_iter, avg_err_in, avg_err_out = experiments(data_size, test_size, num_expriment)

print("The average of E_in: {}".format(avg_err_in))
print("The average of E_out: {}".format(avg_err_out))

The average of E_in: 0.03948
The average of E_out: 0.048839


In [23]:
# Question 7
def experiment2(train_size, test_size, num_exp):
    training_size  = train_size
    testing_size = test_size
    
    avg_iter = 0
    avg_err_in = 0
    avg_err_out = 0
    
    for i in xrange(num_exp):
        p1 = (random.uniform(-1, 1), random.uniform(-1, 1))
        p2 = (random.uniform(-1, 1), random.uniform(-1, 1))
        
        training_set = generate_dataset(training_size)
        testing_set = generate_dataset(testing_size)
        
        # linear regression
        final_weights_lr = train(training_set, p1, p2)
        
        # pla
        iterations, final_weights_pla = pla(training_set, p1, p2, 
                                            weights=final_weights_lr)
        avg_iter += iterations
        
        err_in = test(training_set, p1, p2, final_weights_pla)
        avg_err_in += err_in
        
        err_out = test(testing_set, p1, p2, final_weights_pla)
        avg_err_out += err_out

    
    avg_iter /= float(num_exp)
    avg_err_in /= float(num_exp)
    avg_err_out /= float(num_exp)
    
    return avg_iter, avg_err_in, avg_err_out

In [24]:
data_size = 10
test_size = 10

num_exp = 1000

avg_iter, avg_err_in, avg_err_out = experiment2(data_size, test_size, num_exp)

print("Dataset(N={0}): Average iterations: {1}.".format(data_size, avg_iter))
print("The average of E_in: {}".format(avg_err_in))
print("The average of E_out: {}".format(avg_err_out))

Dataset(N=10): Average iterations: 4.285.
The average of E_in: 0.0
The average of E_out: 0.1008
